In [ ]:
import pandas as pd
from common import *

# Compare iSnobal SWI to USGS discharge gauge at Almont 

## USGS 

Source: https://nwis.waterdata.usgs.gov/co/nwis/dv/?site_no=09112500&agency_cd=USGS&amp;referred_module=sw

In [ ]:
PD_OPTIONS = dict(
    comment='#',
    parse_dates=True,
    index_col=0,
    names=['Date', 'Discharge'],
    header=0,
    sep='\t',
    dtype={'Discharge': np.float64},
    usecols=[2,3],
)
data = pd.read_csv(
    DATA_DIR / 'USGS-SWI-Almont_2022.csv', 
    **PD_OPTIONS
)

data.Discharge = data.Discharge * 60 * 60 * 24

## SNOBAL

In [ ]:
client = start_cluster(8, 24)
client_ip_and_port(client)

In [ ]:
# Convert SNOBAL grid cell from meter to square feet
CELL_AREA = (50.0 * 3.28084)**2
# MM in one foot
FT_IN_MM = 304.8
# Convert to area
CELL_AREA_IN_MM = CELL_AREA / FT_IN_MM

In [ ]:
snobal_em = xr.open_mfdataset(
    (SNOBAL_DIR / 'wy*/erw/*/em.nc').as_posix(),
    data_vars=['SWI'],
    parallel=True,
)
snobal_em.coords['mask'] = (('y', 'x'), cbrfc_zones())
snobal_em['time'] = snobal_em.time + np.timedelta64(2,'h')

In [ ]:
swi = (
    snobal_em.where(snobal_em.mask != 0).SWI * CELL_AREA_IN_MM
).sum(['x', 'y']).compute()

### HRRR MODIS 

In [ ]:
snobal_em_HM = xr.open_mfdataset(
    (SNOBAL_DIR / 'wy*/erw_hrrr_solar_modis/*/em.nc').as_posix(),
    data_vars=['SWI'],
    parallel=True,
)
snobal_em_HM.coords['mask'] = (('y', 'x'), cbrfc_zones())
snobal_em_HM['time'] = snobal_em_HM.time + np.timedelta64(2,'h')

In [ ]:
swi_HM = (
    snobal_em_HM.where(snobal_em_HM.mask != 0).SWI * CELL_AREA_IN_MM
).sum(['x', 'y']).compute()

In [ ]:
client.shutdown()

### Calculate added/removed daily SWE 

### SNOBAL 

### Moving 7-day average SNOBAL vs. USGS

In [ ]:
x_axis_fmt = mdates.DateFormatter('%d-%b-%y')

In [ ]:
fig = plt.figure(dpi=300, figsize=(9, 3))
fig.set_facecolor('whitesmoke')
ax = plt.gca()

swi_HM.rolling(time=7, center=True).mean().plot(ax=ax, label='HRRR-MODIS SWI', color='olive', alpha=0.7, lw=1)
swi.rolling(time=7, center=True).mean().plot(ax=ax, label='Time-Decay SWI', color='peru', alpha=0.7, lw=1, ls='-.')
data.rolling(7, center=True).mean().Discharge.plot(ax=ax, label='USGS Discharge', color='cadetblue', alpha=0.7, lw=1)

ax.xaxis.set_major_formatter(x_axis_fmt)
ax.set_xlim(left=swi_HM.time.min().values, right=np.datetime64('2022-07-01'))
ax.ticklabel_format(axis='y', useMathText=True)
ax.set_xlabel('')

plt.ylabel('7-Day Mean Output (mm)')
plt.legend(
    loc='upper left',
    bbox_to_anchor=(0.2, 1.125),
    ncol=5,
    borderaxespad=0.15, 
    fontsize=8
);

In [ ]:
wy2018 = dict(time=slice("2017-10-01", "2018-09-30"))
wy2019 = dict(time=slice("2018-10-01", "2019-09-30"))
wy2020 = dict(time=slice("2019-10-01", "2020-09-30"))
wy2021 = dict(time=slice("2020-10-01", "2021-09-30"))

In [ ]:
use_hvplot()

In [ ]:
(
    swi.rolling(time=7, center=True).mean().hvplot(label='SMRF SWI', color='goldenrod', alpha=0.7)* \
    swi_HM.rolling(time=7, center=True).mean().hvplot(label='HRRR MODIS SWI', color='crimson', alpha=0.7)* \
    data.rolling(7, center=True).mean().hvplot.line(label='USGS Discharge', color='cadetblue')
).opts(
    title='iSnobal SWI vs USGS Discharge; 7-day moving average', 
    xlabel='Date', 
    ylabel='7-Day Mean Output (mm)', yformatter='%e',
    width=1440, height=800,
    fontsize={'title': 16, 'labels': 14, 'xticks': 14, 'yticks': 14, 'legend': 14}
)